### Descriptive statistics on mortality at Russia

Dataset **deaths.csv** contains absolute number of deaths by Russian region, which are based on death registration information collected by the State Statistic Servise, count all age group and any cauce of deaths for 2015 to 2020 period




In [18]:
# ipython magic
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 5)


In [19]:
import pandas as pd
pd.plotting.register_matplotlib_converters()

In [22]:
data = pd.read_csv('../data/deaths.csv', parse_dates=True)
data.tail(5)

,territory,id_indicator,period,territory.ccube.0,territory.ccube.1,territory.ccube.2,territory.ccube.attr.0,value
6200,10000000,819,28 Feb 2020 00:00:00,Российская Федерация,Дальневосточный федеральный округ,Амурская область,3,803
6201,44000000,819,28 Feb 2020 00:00:00,Российская Федерация,Дальневосточный федеральный округ,Магаданская область,3,104
6202,64000000,819,28 Feb 2020 00:00:00,Российская Федерация,Дальневосточный федеральный округ,Сахалинская область,3,460
6203,99000000,819,28 Feb 2020 00:00:00,Российская Федерация,Дальневосточный федеральный округ,Еврейская автономная область,3,172
6204,77000000,819,28 Feb 2020 00:00:00,Российская Федерация,Дальневосточный федеральный округ,Чукотский автономный округ,3,31


In [21]:
data.drop(columns=['territory',  'id_indicator', 'territory.ccube.0','territory.ccube.attr.0'], inplace=True)
data.rename(columns={'territory.ccube.1':'district', 'territory.ccube.2':'subject', 'value':'deaths'}, inplace=True)
data.tail(5)

,period,district,subject,deaths
6200,28 Feb 2020 00:00:00,Дальневосточный федеральный округ,Амурская область,803
6201,28 Feb 2020 00:00:00,Дальневосточный федеральный округ,Магаданская область,104
6202,28 Feb 2020 00:00:00,Дальневосточный федеральный округ,Сахалинская область,460
6203,28 Feb 2020 00:00:00,Дальневосточный федеральный округ,Еврейская автономная область,172
6204,28 Feb 2020 00:00:00,Дальневосточный федеральный округ,Чукотский автономный округ,31
